In [8]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import json

import os
from typing import List, Tuple, Union


from scipy.io import loadmat,savemat
import matplotlib.pyplot as plt



from statesim.analysis import plot_simulation_results as plot
from statesim.system import pendulum
from statesim.configuration import GenerateConfig
from statesim.simulator import ContinuousSimulator
from statesim.model.statespace import Nonlinear

from crnn.configuration.experiment import load_configuration
from crnn.models.model_io import get_model_from_config
from crnn.models.base import load_model
from crnn.additional_tests import InputOutputStabilityL2, retrieve_additional_test_class, AdditionalTestConfig
from crnn.tracker.io import IoTracker
from crnn.utils import base as utils

import torch
torch.set_default_dtype(torch.double)

from dataclasses import asdict



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

# data_file_name = '/Users/jack/cloud_privat/03_Promotion/_transfer/P-32-joint-lstm/seq/test_output-stability_l2-nonlinear-initial_state-0_M-500_T-10.mat'
config_file_name = '/Users/jack/actuated_pendulum/data/nonlinear-initial_state-0_M-500_T-10/raw/config.json'
result_root_directory = '/Users/jack/cloud_privat/03_Promotion/_transfer/'
m_e_config_file_name = '/Users/jack/cloud_privat/03_Promotion/_transfer/actuated_pendulum.json'

e_m_config = load_configuration(m_e_config_file_name)



In [ ]:
b, num_layer = 1,2
m_e_name = 'P-32-joint-lstm'

model_config = e_m_config.models[m_e_name]
initializer, predictor = get_model_from_config(model_config)
initializer, predictor = (
    load_model(
        model, 
        os.path.join(result_root_directory, m_e_name, utils.get_model_file_name(name, m_e_name))
    ) 
    for model, name in zip([initializer, predictor], ["initializer", "predictor"])
)


test_config = AdditionalTestConfig(epochs=1000, horizon=500, lr=0.01)
test = InputOutputStabilityL2(test_config,predictor, IoTracker())

x0 = (torch.zeros(num_layer,b,predictor.nx),torch.zeros(num_layer,b,predictor.nx))

res = test.test()
gamma_max = res.value
io_data = asdict(res.input_output[0])
print(f'ga: {gamma_max}, norm e_hat: {np.linalg.norm(io_data["e_hat"])}, norm d: {np.linalg.norm(io_data["d"])}')

# io_data = loadmat(data_file_name)

print(np.linalg.norm(io_data['d']))
e_hat, _ = predictor.forward(torch.tensor(io_data['d']), x0)

with open(config_file_name, 'r') as f:
    config = json.load(f)

config = GenerateConfig(**config)
N = int(config.T / config.step_size)

sys = pendulum.ActuatedPendulum()
us = [d_k.reshape(sys.nu, 1) for d_k in io_data['d'][0]]
T = len(us)*config.step_size

sim = ContinuousSimulator(T=T, step_size=config.step_size)
model = Nonlinear(
    sys.state_dynamics,
    sys.output_function,
    sys.nx,
    sys.ny,
    sys.nu
)

result = sim.simulate(
    model=model,
    initial_state=np.array(config.simulator.initial_state).reshape(
        config.system.nx, 1
    ),
    input=us,
    noise_config=config.measurement_noise,
)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(result.t, np.vstack(result.ys), label='e')
ax.plot(result.t, e_hat[0].detach().numpy(), label='e_hat evaluated')
# ax.plot(result.t, io_data['e_hat'][0], label='e_hat predicted')

ax.set_xlabel('Time [s]')
ax.legend()
ax.grid()

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(result.t, np.vstack(result.us), label='u')
ax.grid()
ax.legend()
ax.set_xlabel('Time [s]')

0/1000: ga: 3.82, norm e_hat: 3.82, norm d: 1.01
1/1000: ga: 4.04, norm e_hat: 4.10, norm d: 1.02
2/1000: ga: 4.07, norm e_hat: 4.16, norm d: 1.01
4/1000: ga: 4.15, norm e_hat: 4.09, norm d: 0.96
8/1000: ga: 4.18, norm e_hat: 3.88, norm d: 0.92
11/1000: ga: 4.36, norm e_hat: 3.97, norm d: 0.90
12/1000: ga: 4.70, norm e_hat: 4.23, norm d: 0.89
15/1000: ga: 4.77, norm e_hat: 4.08, norm d: 0.84
17/1000: ga: 4.80, norm e_hat: 3.99, norm d: 0.82
21/1000: ga: 4.95, norm e_hat: 3.93, norm d: 0.78
23/1000: ga: 5.04, norm e_hat: 3.87, norm d: 0.76
24/1000: ga: 5.15, norm e_hat: 3.89, norm d: 0.74
25/1000: ga: 5.42, norm e_hat: 4.03, norm d: 0.73
30/1000: ga: 5.58, norm e_hat: 3.83, norm d: 0.67
31/1000: ga: 5.70, norm e_hat: 3.84, norm d: 0.66
33/1000: ga: 5.82, norm e_hat: 3.76, norm d: 0.63
34/1000: ga: 5.96, norm e_hat: 3.77, norm d: 0.62
35/1000: ga: 6.02, norm e_hat: 3.74, norm d: 0.61
36/1000: ga: 6.07, norm e_hat: 3.69, norm d: 0.60
37/1000: ga: 6.36, norm e_hat: 3.79, norm d: 0.58
38/10